In [ ]:
import numpy as np 
import pandas as pd 

import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay
from glob import glob
import re
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Read Data

In [ ]:
base_dir = "/kaggle/input/vehicle-detection-image-set/data"

In [ ]:
label = []       #  Assign all label
images_path = [] # Get all images path
all_images = []  # Check all images name
idx = 0          # index
img_label = []   # Assign image label



for directory in os.listdir(base_dir):
    label.append(directory)
    for img in os.listdir(os.path.join(base_dir,directory)):
        if len(re.findall('.png',img.lower())) != 0 or len(re.findall('.jpg',img.lower())) != 0 or len(re.findall('.jpeg',img.lower())) != 0:
            img_path = os.path.join(os.path.join(base_dir,directory),img)
            all_images.append(img)
            images_path.append(img_path)
            img_label.append(directory)

In [ ]:
img_shape = cv2.imread(images_path[0])

print(f"Labels : { label}")
print(f"Total Images : {len(all_images)}")
print(f" Images shape : {img_shape.shape}")

In [ ]:
data=pd.DataFrame({"image":images_path,"Label":img_label})
data

## show some data

In [ ]:
random_img = data.sample(10).reset_index()
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(25, 25),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(random_img.image[i]))
    ax.set_title(random_img.Label[i], color='blue', fontsize=14)
    ax.axis('off')
plt.tight_layout(pad=3.0)
plt.show()

## check the balance 

In [ ]:
data_label = data['Label'].value_counts()
#print(data_label.index)
plt.figure()
sns.barplot(x = data_label.index, y = data_label)
plt.xlabel("Type of Data")
plt.ylabel("Total Data")
plt.title("Bar Chart Of Dataset")
plt.show()
print(f"Total Images class [0]:{data_label[0]}")
print(f"Total Images class [1]:{data_label[1]}")

## split data

In [ ]:
X_train, X_test = train_test_split(data,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)

# Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.1,
                                    rotation_range = 30,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    horizontal_flip = True,
                                    validation_split = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_dataframe(
    dataframe = X_train,
    x_col = "image",
    y_col = "Label",
    target_size = (64, 64),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 2,
    subset = "training")

validation_set = train_datagen.flow_from_dataframe(
    dataframe = X_train,
    x_col = "image",
    y_col = "Label",
    target_size = (64, 64),
    color_mode ="rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 2,
    subset = "validation")

test_set = test_datagen.flow_from_dataframe(
    dataframe = X_test,
    x_col = "image",
    y_col = "Label",
    target_size = (64, 64),
    color_mode ="rgb",
    class_mode = "categorical",
    shuffle = False,
    batch_size = 32)

In [ ]:
x=img_to_array(img)
x.shape

In [ ]:
model = tf.keras.Sequential()
#first layer of model
model.add(layers.Conv2D(32,(3,3),input_shape=img_shape.shape))
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))
#second layer of model
model.add(layers.Conv2D(64,(3,3)))
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))





model.add(layers.Flatten())
model.add(layers.Dense(1024))#input neurons
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(label)))#output layer
model.add(layers.Activation("softmax"))

In [ ]:
model.summary()

In [ ]:
optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    epsilon=1e-07,
    name="RMSprop"
)

In [ ]:
#Compile Model
model.compile(optimizer = optimizer, loss='categorical_crossentropy', 
             metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
history = model.fit(training_set, epochs=10,validation_data = validation_set, callbacks=[early_stopping],batch_size=32)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#evaluate model
test_score = model.evaluate(test_set)

In [ ]:
predicted_label = model.predict(test_set)
predicted_label = np.argmax(predicted_label,axis=1)

In [ ]:
test_datas = test_set.filenames
labels_data = test_set.labels

fig = plt.figure(figsize=(15,15))

for i in range(8):
    if labels_data[i] == 0:
        test_label = "Non Vehicle"
    else:
        test_label = "Vehicle"
    if predicted_label[i] == 0:
        predict_lab = "Non Vehicle"
    else :
        predict_lab = "Vehicle"
    label = "Label : "+ test_label +" Pred: "+predict_lab
    
    fig.add_subplot(4,2,i+1).set_title(label)
    img = cv2.imread(test_datas[i])
    plt.imshow(img)

# VGG-16

In [ ]:
vgg= VGG16()

In [ ]:
print(len(vgg.layers))

In [ ]:
vgg.trainable = False

In [ ]:
vgg.summary() 

In [ ]:
#remove the prediction layer in vgg-model
model_vgg = tf.keras.Sequential()
for i in range(len(vgg.layers)-1):
    model_vgg.add(vgg.layers[i])
print(model_vgg.summary())

In [ ]:
#add our prediction layer 
from tensorflow.keras import layers
model_vgg.add(layers.Dense(len(label),activation="softmax"))

In [ ]:
model_vgg.summary()

### handel the data to suitble with vgg-model

In [ ]:
training_set_vgg = train_datagen.flow_from_dataframe(
    dataframe = X_train,
    x_col = "image",
    y_col = "Label",
    target_size = (244, 244),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 2,
    subset = "training")

validation_set_vgg = train_datagen.flow_from_dataframe(
    dataframe = X_train,
    x_col = "image",
    y_col = "Label",
    target_size = (224, 224),
    color_mode ="rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 2,
    subset = "validation")

test_set_vgg = test_datagen.flow_from_dataframe(
    dataframe = X_test,
    x_col = "image",
    y_col = "Label",
    target_size = (224, 224),
    color_mode ="rgb",
    class_mode = "categorical",
    shuffle = False,
    batch_size = 32)


In [ ]:
model_vgg.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

In [ ]:
history = model_vgg.fit(training_set_vgg, epochs=10,validation_data = validation_set_vgg,batch_size=32)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_score = model_vgg.evaluate(test_set_vgg)

# ResNet

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
    )

In [ ]:
resnet_model = tf.keras.applications.resnet50.ResNet50(input_shape=img_shape.shape,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
resnet_model.trainable = False

In [ ]:
resnet_model.summary()

In [ ]:
x = resnet_model.output

# Add a global average pooling layer
x = GlobalAveragePooling2D()(x)

# Add your custom dense layer
predictions = Dense(2, activation='softmax')(x)  

# Create the final model
model_res = Model(inputs=resnet_model.input, outputs=predictions)

In [ ]:
model_res.summary()

In [ ]:
model_res.compile(loss="categorical_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"])

In [ ]:
history = model_res.fit(training_set, epochs=10,validation_data = validation_set, callbacks=[early_stopping],batch_size=32)

In [ ]:
test_score = model_res.evaluate(test_set)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()